Client to run jsoniq queries in rumble server.
For python client see : https://github.com/dmwm/CMSKubernetes/tree/master/docker/rumble/client_scripts

In [2]:
import requests
import json
import ast
from IPython.core.magic import register_line_cell_magic

# Rumble server endpoint in K8S test cluster. Max output size is 1.000.000 records
server = "http://test-cluster-jnbxujghdusq-node-0.cern.ch/jsoniq?materialization-cap=1000000"
@register_line_cell_magic
def rumble(line, cell=None):
    global server, output_path
    if cell is None:
        data = line
    else:
        data = cell

    response = json.loads(requests.post(server, data=data).text)
    
    if 'warning' in response:
        print(ast.literal_eval(json.dumps(response['warning'])))
    if 'values' in response:
        for e in response['values']:
            print(ast.literal_eval(json.dumps(e)))
    elif 'error-message' in response:
        return response['error-message']
    else:
        return response

In [3]:
%rumble 1 + 1

2


In [4]:
%%time
%%rumble
for $i in parallelize(5 to 10)
where $i mod 9 eq 0
return $i

9
CPU times: user 47.7 ms, sys: 14.9 ms, total: 62.6 ms
Wall time: 46 s


In [9]:
%%time
%%rumble
subsequence(
  for $doc in json-file("hdfs://analytix/project/monitoring/archive/wmarchive/raw/metric/2020/09/15/")
  return $doc.data.wmaid,
  1,
  10
)

420fc15e6d8242cba1f17478af5e5a4f
f7ada3ce4cd844749ddb719d91f60956
77fc8211f70a48248d62304989435398
e89af265ecc84a82a3c10404a7f2fd5d
83037e80949249328d59092964fbc12e
e5b709fa05974cfaa58eb5cd2e691393
6adbe57eb17c473a8de09a84b28e0c31
bb7e472d5a934c6392a4a802b2d0d058
a5cff64db7be41f089536ce066347ac7
f2b73f0062064df4ada4b96bcab7a24a
CPU times: user 124 ms, sys: 58.5 ms, total: 183 ms
Wall time: 2min 37s
